# 추천

### 최근접 이웃 협업 필터링
- COLLABORATIVE FILTERING:  잠재요인 협업 필터링 , 협업기반의 필터링
- CONTENT-BASED FILTERING: 컨텐츠(아이템) 기반의 추천,


- 최근접 이웃 협업 필터링
  - 아이템기반
  - 유저기반

- 최근접 이웃 협업 필터링
  - 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식을 기반으로 추천
  - User-Item Matrix Concept
  - User사용자가 평가한 다른 아이템을 기반으로 사용자가 평가하지 않은 아이템의 예측 평가를 도출하는 방식
### 사용자 기반(User-User) 협업 필터링
- 나와 다른 사용자 간의 유사도를 측정하여 유사도가 높은 다른 사용자 N명을 추출하여 그들이 선호하는 아이템을 추천하는 방식

### 아이템 기반(item-item) 협업 필터링
- 나와 동일한 아이템을 사용해본 다른 사용자들과 평점 분포가 비슷한 다른 아이템을 추천하는 방식
- 일반적으로 사용자 기반보다는 아이템 가반 협업 필터링이 정확도가 더 높다.

- 잠재 요인 협업 필터링
  - 사용자_아이템 평저 메트릭스 속에 숨어 있는 잠재 요인을 추출해서 추천 예측을 하는 기법
  - 대규모 다차원 행렬인 사용자_아이템 행렬 데이터를 고윳값 분해와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인을 추출한다.
  - 사용자 X 아이템 행렬 = (사용자 X 잠재요인 행렬) . (아이템 X 잠재요인 행렬).T 로 분해 후 두행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 생성하여 사용자가 아직 평점을 부여하지 않은 아이ㅔㅁ에 대한 예측 평점을 생성하는 것
  - `잠재요인`인 어떤것인지는 명확히 정의할 수 없다.
  - 만약, 영화 평점 기반의 사용자0아이템 행렬 데이터라면 영화가 가지는 장르별 특성 선호도로 가정할 수 있다.
  

### 특이값 분해(SVD) 싱귤러 벡터 디포지션
- SVD란 m x n 행렬일 떄 , 다음과 같이 3개의 행렬의 곱으로 분해하는 행렬 분해 방법 중 하나이다.
- SVD는 NaN값이 없는 행렬에만 적용할 수 있다. 하지만 R행렬에(시용자-영화 평점 행렬)는 아직 평점되지 않은 많은 Null값이 있기 떄문에 P와 Q행렬을 일반적인 SVD 방식으로 분해할 수 없다.
- 이러한 경우에는 확률적 경사 하강법 방식을 이용해 SVD를 수행한다.

kaggle
https://www.imdb.com/

# 장르 속성을 이용한 영화 콘텐츠 기반 필터링
- TMDB 5000 데이터 셋: 영화 데이터 정보 사이트인 IMDB.com의 영화 중 주요 영화 5000개에 대한 메타 정보를 가공해서 kaggle에서 제공하는 데이터 셋 
- https://www.kaggle.com/code

### 1.1 데이터 로딩 및 가공

In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

movies = pd.read_csv('./tmdb_5000_movies.csv')
print(movies.shape)
movies.head(2)


(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


- 분석에 사용할 주요 컬럼 추출
  - id, title, genres, vote_average(평균 평점), vote_count(평점 투표수), popularity(영화 인기도), keyword, overview(영화 개요)

In [9]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [15]:
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."


In [20]:
movies_df['genres'].values[0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

### eval() 와 literal_eval() 의 차이
- eval        : 문자 형태로 되어있는 표현식을 실행하는 함수로, 함수나 객체도 실행가능
- literal_eval: 파이선에서 제공하는 기본 데이터 타입 정도만 변환해주는 용도로 사용 가능

- literal_eval() 함수를 통해 genres 와 keywords 컬럼의 값을 List객체로 변환

In [13]:
s = '(1+2) * 3'
eval(s)

9

In [14]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

- 장르와 키워드 컬럼의 값 중 name 키 값만 원소로 추출하여 리스트로 생성

In [19]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])